<think>
Vấn đề localhost không thể gọi được backend có thể liên quan đến chính sách bảo mật CORS (Cross-Origin Resource Sharing). Khi bạn chạy frontend trên localhost (http://127.0.0.1:5500) và cố gắng gọi API trên một domain khác (http://103.253.20.13:25050), trình duyệt sẽ chặn request nếu backend không có cấu hình CORS phù hợp.

Một số nguyên nhân có thể:

1. **CORS không được cấu hình trên backend**: Backend cần cấu hình CORS để cho phép request từ domain khác.
2. **Giao thức khác nhau**: HTTP vs HTTPS cũng có thể gây ra vấn đề CORS.
3. **Proxy trong Docker**: Khi bạn chạy trong Docker, có thể có proxy hoặc mạng nội bộ cho phép kết nối mà không cần CORS.

Cách giải quyết:

1. **Cấu hình CORS trên backend**: Thêm header cho phép truy cập từ tất cả nguồn gốc (`Access-Control-Allow-Origin: *`).
2. **Sử dụng proxy trên frontend**: Thêm proxy vào cấu hình development của frontend.
3. **Sử dụng extension tắt CORS trong trình duyệt**: Chỉ cho mục đích phát triển.
4. **Chạy frontend trong cùng domain với backend**: Đây là cách Docker làm.

Tôi sẽ đề xuất giải pháp dựa trên CORS và proxy.
</think>

# Sửa lỗi localhost không call được backend

Vấn đề bạn đang gặp là do chính sách CORS (Cross-Origin Resource Sharing) của trình duyệt. Khi chạy trên localhost, trình duyệt chặn các request đến domain khác (103.253.20.13) nếu backend không cấu hình cho phép CORS.

## Có 2 giải pháp:

### 1. Sử dụng CORS Proxy trong môi trường phát triển

````javascript:src/frontend/index.html
// Cập nhật phương thức gọi API với CORS proxy cho môi trường development
const startSimulation = async () => {
    // ... code giữ nguyên ...
    
    try {
        const apiCalls = selectedPrompts.map(async (prompt) => {
            const requestData = {
                bot_id: botId,
                user_prompt: prompt.content,
                max_turns: maxTurns
            };
            
            // Kiểm tra nếu đang chạy trên localhost thì dùng CORS proxy
            let apiEndpoint = 'http://103.253.20.13:25050/simulate';
            
            // Thêm CORS proxy nếu đang chạy trên localhost
            if (window.location.hostname === 'localhost' || window.location.hostname === '127.0.0.1') {
                apiEndpoint = `https://cors-anywhere.herokuapp.com/${apiEndpoint}`;
                // Hoặc apiEndpoint = `https://api.allorigins.win/raw?url=${encodeURIComponent(apiEndpoint)}`;
            }
            
            const response = await fetch(apiEndpoint, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify(requestData),
            });
            
            // ... code tiếp theo giữ nguyên ...
````






### 2. Chạy một local CORS proxy

```bash
# Cài đặt cors-anywhere
npm install -g cors-anywhere

# Chạy local CORS proxy
cors-anywhere
```

Sau đó sửa API URL thành:

````javascript:src/frontend/index.html
// Sử dụng local CORS proxy
let apiEndpoint = 'http://103.253.20.13:25050/simulate';
            
// Thêm local CORS proxy nếu đang chạy trên localhost
if (window.location.hostname === 'localhost' || window.location.hostname === '127.0.0.1') {
    apiEndpoint = `http://localhost:8080/${apiEndpoint}`;
}
````






### 3. Giải pháp nhanh: Sử dụng extension tắt CORS

Cài đặt extension như "CORS Unblock" hoặc "Allow CORS" trong trình duyệt Chrome/Firefox để tạm thời tắt kiểm tra CORS trong quá trình phát triển.

### Tại sao Docker hoạt động?

Khi chạy trong Docker, frontend và backend có thể được cấu hình trong cùng một mạng nội bộ hoặc Nginx đã xử lý vấn đề CORS bằng cách thêm các header phù hợp.

Tốt nhất là giữ nguyên cấu hình Docker hiện tại vì nó đã hoạt động tốt, và chỉ sử dụng một trong những giải pháp trên khi cần phát triển trên localhost.
